# IMPORT DATA, PACKAGES, AND FUNCTIONS

## import the relevant functions and packages

In [1]:
import pandas as pd
import numpy as np
from test_case_generator_functions import special_characters, alphabet

# TEST CASE TYPE REFERENCE INFORMATION

In [2]:
uid_list = list(range(210, 213))
entity_type_list = ['Country', 'Entity', 'Individual']

uid = 'UID-212' 
theme = 'Names where name parts are Modified'
category = 'Character replaced by Number and Special Character'
sub_category = '1 Letter replaced by number and 1 letter replaced by special character'
entity_type = 'Individual'

## download the OFAC list from the web

In [3]:
ofac_list_download = pd.read_csv('https://www.treasury.gov/ofac/downloads/sdn.csv', header=None)
ofac_list_download

,0,1,2,3,4,5,6,7,8,9,10,11
0,36,AEROCARIBBEAN AIRLINES,-0-,CUBA,-0-,-0-,-0-,-0-,-0-,-0-,-0-,-0-
1,173,"ANGLO-CARIBBEAN CO., LTD.",-0-,CUBA,-0-,-0-,-0-,-0-,-0-,-0-,-0-,-0-
2,306,BANCO NACIONAL DE CUBA,-0-,CUBA,-0-,-0-,-0-,-0-,-0-,-0-,-0-,a.k.a. 'BNC'.
3,424,BOUTIQUE LA MAISON,-0-,CUBA,-0-,-0-,-0-,-0-,-0-,-0-,-0-,-0-
4,475,CASA DE CUBA,-0-,CUBA,-0-,-0-,-0-,-0-,-0-,-0-,-0-,-0-
...,...,...,...,...,...,...,...,...,...,...,...,...
10563,39257,"ABNOUSH, Salar",individual,IRAN-HR,-0-,-0-,-0-,-0-,-0-,-0-,-0-,"DOB 02 May 1962; POB Hamedan, Iran; nationalit..."
10564,39258,IRAN'S MORALITY POLICE,-0-,IRAN-HR,-0-,-0-,-0-,-0-,-0-,-0-,-0-,Additional Sanctions Information - Subject to ...
10565,39259,"MIRZAEI, Haj Ahmad",individual,IRAN-HR,Colonel,-0-,-0-,-0-,-0-,-0-,-0-,DOB 09 Feb 1957; nationality Iran; Additional ...
10566,39260,"ROSTAMI CHESHMEH GACHI, Mohammad",individual,IRAN-HR,General,-0-,-0-,-0-,-0-,-0-,-0-,DOB 1976 to 1977; nationality Iran; Additional...


In [4]:
ofac_list = ofac_list_download[[0,1,2]]
ofac_list.columns = ['uid', 'name', 'entity_type']
ofac_list

,uid,name,entity_type
0,36,AEROCARIBBEAN AIRLINES,-0-
1,173,"ANGLO-CARIBBEAN CO., LTD.",-0-
2,306,BANCO NACIONAL DE CUBA,-0-
3,424,BOUTIQUE LA MAISON,-0-
4,475,CASA DE CUBA,-0-
...,...,...,...
10563,39257,"ABNOUSH, Salar",individual
10564,39258,IRAN'S MORALITY POLICE,-0-
10565,39259,"MIRZAEI, Haj Ahmad",individual
10566,39260,"ROSTAMI CHESHMEH GACHI, Mohammad",individual


In [5]:
ofac_list['entity_type'].value_counts()

individual    5401
-0-           4366
vessel         514
aircraft       286
Name: entity_type, dtype: int64

# FILTER FOR THE REQUIREMENTS OF THE TEST CASE TYPE

## filter for the requirements of the specific test case type

In [5]:
ofac_list_filtered = ofac_list[(ofac_list.entity_type == 'individual')] # only evaluate individuals
ofac_list_filtered

,uid,name,entity_type
53,2674,"ABBAS, Abu",individual
54,2675,"AL RAHMAN, Shaykh Umar Abd",individual
55,2676,"AL ZAWAHIRI, Dr. Ayman",individual
56,2677,"AL-ZOMOR, Abboud Abdul Latif Hassan",individual
57,2678,"AWDA, Abd Al Aziz",individual
...,...,...,...
10555,39211,"SHELESTENKO, Hennadiy Oleksandrovych",individual
10556,39212,"KOBETS, Oleksandr Yuriyovych",individual
10557,39213,"SEMENCHEV, Ihor Ihorovych",individual
10558,39214,"KUZ'MYCH, Tetyana Oleksandrivna",individual


## randomly choose 10 rows

In [6]:
ofac_list_sampled = ofac_list_filtered.sample(n = 10)
ofac_list_sampled

,uid,name,entity_type
9105,34658,"FRADKOV, Petr Mikhailovich",individual
7000,26683,"ALVARES, Carlos",individual
600,7828,"AMDOUNI, Mehrez",individual
6690,26078,"TUBAIGY, Salah Muhammed A.",individual
1100,9390,"ALTAMIRANO LOPEZ, Hector",individual
3678,16993,"SCHIAVONE, Francesco",individual
9508,35491,"TSED, Nikolay Grigorevich",individual
2197,12088,"PELAEZ LOPEZ, John Jairo",individual
6002,24519,"AL-SAYYID, Ibrahim Amin",individual
5363,22863,"AL-MANSUR, Salim Mustafa Muhammad",individual


# CREATE THE TEST CASES

## create blank final test cases table

In [7]:
final_test_cases = pd.DataFrame(columns=['UID', 'Theme','Category','Sub-category','Entity-Type','Test Case ID' , 'OFAC List UID', 'Original Name','Test Case Name'])
final_test_cases

,UID,Theme,Category,Sub-category,Entity-Type,Test Case ID,OFAC List UID,Original Name,Test Case Name


## run loop to generate the test cases

In [8]:
for index, row in ofac_list_sampled.iterrows():
    original_name = row['name']

    replace_list = [] # get index of alphabets in each name
    for i in range(len(row['name'])):
        if row['name'][i].upper() in alphabet():
            replace_list.append(i)
    
    replace_id = np.random.choice(replace_list) # replace 1 random alphabet with special character
    temp_name = original_name[:replace_id] + np.random.choice(special_characters()) + original_name[replace_id + 1:]
    replace_list.remove(replace_id)
    
    replace_id = np.random.choice(replace_list) # replace 1 random alphabet with number
    final_test_name = temp_name[:replace_id] + str(np.random.randint(10)) + temp_name[replace_id + 1:]
    
    final_test_cases.loc[len(final_test_cases)] = [uid, theme, category, sub_category, entity_type, uid + ' - ' + str(index), row['uid'], row['name'], final_test_name] # append to the dataframe

final_test_cases

,UID,Theme,Category,Sub-category,Entity-Type,Test Case ID,OFAC List UID,Original Name,Test Case Name
0,UID-212,Names where name parts are Modified,Character replaced by Number and Special Chara...,1 Letter replaced by number and 1 letter repla...,Individual,UID-212 - 9105,34658,"FRADKOV, Petr Mikhailovich","F1ADKOV, Petr Mikhailovi()h"
1,UID-212,Names where name parts are Modified,Character replaced by Number and Special Chara...,1 Letter replaced by number and 1 letter repla...,Individual,UID-212 - 7000,26683,"ALVARES, Carlos","ALVARE6, C#rlos"
2,UID-212,Names where name parts are Modified,Character replaced by Number and Special Chara...,1 Letter replaced by number and 1 letter repla...,Individual,UID-212 - 600,7828,"AMDOUNI, Mehrez","AM(1OUNI, Mehrez"
3,UID-212,Names where name parts are Modified,Character replaced by Number and Special Chara...,1 Letter replaced by number and 1 letter repla...,Individual,UID-212 - 6690,26078,"TUBAIGY, Salah Muhammed A.","1UBAIGY, Salah Mu)ammed A."
4,UID-212,Names where name parts are Modified,Character replaced by Number and Special Chara...,1 Letter replaced by number and 1 letter repla...,Individual,UID-212 - 1100,9390,"ALTAMIRANO LOPEZ, Hector","ALTAMIRANO 0OPEZ, Hec^or"
5,UID-212,Names where name parts are Modified,Character replaced by Number and Special Chara...,1 Letter replaced by number and 1 letter repla...,Individual,UID-212 - 3678,16993,"SCHIAVONE, Francesco","S(HIAVONE, Frances7o"
6,UID-212,Names where name parts are Modified,Character replaced by Number and Special Chara...,1 Letter replaced by number and 1 letter repla...,Individual,UID-212 - 9508,35491,"TSED, Nikolay Grigorevich","TSED, #ikolay Grigorevic1"
7,UID-212,Names where name parts are Modified,Character replaced by Number and Special Chara...,1 Letter replaced by number and 1 letter repla...,Individual,UID-212 - 2197,12088,"PELAEZ LOPEZ, John Jairo","PELAEZ LOPEZ, John Ja&7o"
8,UID-212,Names where name parts are Modified,Character replaced by Number and Special Chara...,1 Letter replaced by number and 1 letter repla...,Individual,UID-212 - 6002,24519,"AL-SAYYID, Ibrahim Amin","AL-SAYYID, Ibrah3m Am#n"
9,UID-212,Names where name parts are Modified,Character replaced by Number and Special Chara...,1 Letter replaced by number and 1 letter repla...,Individual,UID-212 - 5363,22863,"AL-MANSUR, Salim Mustafa Muhammad","AL-MANSUR, Salim Mustafa @uhamma8"
